# Gather face photos for training

In [1]:
import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

img = cv2.imread('ali.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [4]:
'''
This cell will turn on webcam and take photos, cropping a face if detected, 
and save it to directory ./datasets/name/name-x.png. 
'''
TEAMMATE_NAME = 'ali'  # Insert name here!!
# Initialize Webcam
cap = cv2.VideoCapture(0)

#Load Haarcascade Frontal Face Classifier
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

#Function returns cropped face
def face_extractor(photo):
    gray_photo = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_photo, 1.1, 5)
    
    if faces is ():
        return None
    
    else:
        # Crop all faces found
        for (x,y,w,h) in faces:
            cropped_face = photo[y:y+h, x:x+w]
        
        return cropped_face


count = 0

# Collect 500 samples of your face from webcam input
while True:
    status,photo = cap.read()
    
    if face_extractor(photo) is not None:
        count += 1
        face = cv2.resize(face_extractor(photo), (224, 224)) #Save imgs as 224x224
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './datasets/' + TEAMMATE_NAME + '/' + TEAMMATE_NAME + '-' + str(count) + '.png' #saves images to ./datasets/ali/ali-x.png
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        pass

    if cv2.waitKey(1) == 13 or count == 500: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()    
cv2.waitKey(1)
print("Collecting Samples Complete")

# Precomputing Features

In [2]:
from keras_vggface.vggface import VGGFace
from keras.engine import Model
from keras import models
from keras import layers
from keras.layers import Input
from keras.preprocessing import image
import numpy as np
from keras_vggface import utils
import scipy.spatial as spatial
import cv2
import os
import glob
import pickle



# Helper method to save byte stream of name/features
def pickle_stuff(filename, stuff):
    save_stuff = open(filename, "wb")
    pickle.dump(stuff, save_stuff)
    save_stuff.close()

Using TensorFlow backend.


In [3]:
def precompute_features():
    
    # Load VGGFace for computing face feature vectors
    resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3),
                                pooling='avg')  # pooling: None, avg or max
    
    
    def image2x(image_path):
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = utils.preprocess_input(x, version=1)  # or version=2
        return x

    # Method to calculate mean features map on folder of person images (labeled name)
    def cal_mean_feature(image_folder):
        face_images = list(glob.iglob(os.path.join(image_folder, '*.*')))
        print(str(len(face_images)))
        def chunks(l, n):
            """Yield successive n-sized chunks from l."""
            for i in range(0, len(l), n):
                yield l[i:i + n]

        batch_size = 32
        face_images_chunks = chunks(face_images, batch_size)
        fvecs = None
        for face_images_chunk in face_images_chunks:
            images = np.concatenate([image2x(face_image) for face_image in face_images_chunk])
            print("image len: " + str(len(images)))
            batch_fvecs = resnet50_features.predict(images) #Use VGGFace model to get feature map on images
            
            if fvecs is None: 
                fvecs = batch_fvecs
            else:
                fvecs = np.append(fvecs, batch_fvecs, axis=0) #Append all feature vectors
        return np.array(fvecs).sum(axis=0) / len(fvecs) #Return mean feature map as np array
    

    #FACE_IMAGES_FOLDER = "./datasets/"
    DATASETS = "./datasets/"
    #VIDEOS_FOLDER =
    
    #extractor = FaceExtractor()
    #folders = list(glob.iglob(os.path.join(VIDEOS_FOLDER, '*')))
    folders = list(glob.iglob(os.path.join(DATASETS, '*')))
    os.makedirs(DATASETS, exist_ok=True)
    names = [os.path.basename(folder) for folder in folders]
    
    '''
    for i, folder in enumerate(folders):
        name = names[i]
        videos = list(glob.iglob(os.path.join(folder, '*.*')))
        save_folder = os.path.join(FACE_IMAGES_FOLDER, name)
        print(save_folder)
        os.makedirs(save_folder, exist_ok=True)
        for video in videos:
            extractor.extract_faces(video, save_folder)
    '''
    
    # for each folder of a teammate
    # send image folder to cal_mean_feature() to calc feature map
    # save {"name": teammate_name, "features": mean_feature_map}
    # pickle it ()
    precompute_features = []
    for i, folder in enumerate(folders):
        name = names[i]
        save_folder = os.path.join(DATASETS, name)
        print("save folder: " + str(save_folder))
        
        mean_features = cal_mean_feature(image_folder=save_folder)
        precompute_features.append({"name": name, "features": mean_features})

    # https://stackoverflow.com/questions/8968884/python-serialization-why-pickle
    pickle_stuff("./precompute_features.pickle", precompute_features)

#if __name__ == "__main__":
#    main()

In [4]:
precompute_features()

save folder: ./datasets/alex
285
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 29
save folder: ./datasets/tej
295
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 7
save folder: ./datasets/daniel
271
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 15
save folder: ./datasets/ali
492
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 32
image len: 12


# Face Identifying Engine

In [5]:
# Helper method to load byte stream of name/features map
def load_stuff(filename):
    saved_stuff = open(filename, "rb")
    stuff = pickle.load(saved_stuff)
    saved_stuff.close()
    return stuff

In [6]:
# Main class for new face identification

class FaceIdentify(object):
    """
    Singleton class for real time face identification
    """
    CASE_PATH = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

    def __new__(cls, precompute_features_file=None):
        if not hasattr(cls, 'instance'):
            cls.instance = super(FaceIdentify, cls).__new__(cls)
        return cls.instance

    def __init__(self, precompute_features_file=None):
        self.face_size = 224
        self.precompute_features_map = load_stuff(precompute_features_file)
        print("Loading VGG Face model...")
        self.model = VGGFace(model='resnet50',
                             include_top=False,
                             input_shape=(224, 224, 3),
                             pooling='avg')  # pooling: None, avg or max
        print("Loading VGG Face model done")
        
        
    @classmethod
    def draw_label(cls, image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
                   font_scale=1, thickness=2):
        size = cv2.getTextSize(label, font, font_scale, thickness)[0]
        x, y = point
        cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
        cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)

    def crop_face(self, imgarray, section, margin=20, size=224):
        """
        :param imgarray: full image
        :param section: face detected area (x, y, w, h)
        :param margin: add some margin to the face detected area to include a full head
        :param size: the result image resolution with be (size x size)
        :return: resized image in numpy array with shape (size x size x 3)
        """
        img_h, img_w, _ = imgarray.shape
        if section is None:
            section = [0, 0, img_w, img_h]
        (x, y, w, h) = section
        margin = int(min(w, h) * margin / 100)
        x_a = x - margin
        y_a = y - margin
        x_b = x + w + margin
        y_b = y + h + margin
        if x_a < 0:
            x_b = min(x_b - x_a, img_w - 1)
            x_a = 0
        if y_a < 0:
            y_b = min(y_b - y_a, img_h - 1)
            y_a = 0
        if x_b > img_w:
            x_a = max(x_a - (x_b - img_w), 0)
            x_b = img_w
        if y_b > img_h:
            y_a = max(y_a - (y_b - img_h), 0)
            y_b = img_h
        cropped = imgarray[y_a: y_b, x_a: x_b]
        resized_img = cv2.resize(cropped, (size, size), interpolation=cv2.INTER_AREA)
        resized_img = np.array(resized_img)
        return resized_img, (x_a, y_a, x_b - x_a, y_b - y_a)
    
    
    # This is the big one, takes features of found face and compares it to mean features map
    
    def identify_face(self, features, threshold=100):
        distances = []
        for person in self.precompute_features_map:
            person_features = person.get("features")
            distance = spatial.distance.euclidean(person_features, features) #Compare by Euclidean distance of feature map using scipy spatial
            distances.append(distance)
        min_distance_value = min(distances)
        min_distance_index = distances.index(min_distance_value) 
        if min_distance_value < threshold: # Get min distance, if less than a threshold, return that name
            return self.precompute_features_map[min_distance_index].get("name")
        else: #Else reject
            return "?"
        
    def detect_face(self):
        face_cascade = cv2.CascadeClassifier(self.CASE_PATH)

        # 0 means the default video capture device in OS
        video_capture = cv2.VideoCapture(0)
        # infinite loop, break by key ESC
        while True:
            if not video_capture.isOpened():
                sleep(5)
            # Capture frame-by-frame
            ret, frame = video_capture.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(
                gray,
                scaleFactor=1.2,
                minNeighbors=10,
                minSize=(64, 64)
            )
            # placeholder for cropped faces
            face_imgs = np.empty((len(faces), self.face_size, self.face_size, 3))
            for i, face in enumerate(faces):
                face_img, cropped = self.crop_face(frame, face, margin=10, size=self.face_size)
                (x, y, w, h) = cropped
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 200, 0), 2)
                face_imgs[i, :, :, :] = face_img
            if len(face_imgs) > 0:
                # generate features for each face
                features_faces = self.model.predict(face_imgs)
                predicted_names = [self.identify_face(features_face) for features_face in features_faces]
            # draw results
            for i, face in enumerate(faces):
                label = "{}".format(predicted_names[i])
                self.draw_label(frame, (face[0], face[1]), label)

            cv2.imshow('Keras Faces', frame)
            if cv2.waitKey(5) == 27:  # ESC key press
                break
        # When everything is done, release the capture
        video_capture.release()
        cv2.destroyAllWindows()


In [7]:
def main():
    face = FaceIdentify(precompute_features_file="./precompute_features.pickle")
    face.detect_face()

if __name__ == "__main__":
    main()

Loading VGG Face model...
Loading VGG Face model done
